In [1]:
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))
sys.path.append(os.path.abspath('../../dashgrid'))
sys.path.append(os.path.abspath('../../dashgrid/dashgrid'))
from dashgrid import dgrid
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output,State
import dash_table
import pandas as pd
import numpy as np
from dashgrid import dgrid


In [2]:
def create_dt_div(df_in=None,
                  editable_columns_in=None,title='Dash Table'):
    editable_columns = [] if editable_columns_in is None else editable_columns_in
#     datatable_id = f'{html_id}_datatable'
    dt = dash_table.DataTable(
        page_current= 0,
        page_size= 100,
        page_action='native',
        sort_action='native',
        filter_action='none', # 'fe',
#             content_style='grow',
        style_cell_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ] + [
            {
                'if': {'column_id': c},
                'textAlign': 'left',
            } for c in ['symbol', 'underlying']
        ],

        style_as_list_view=False,
        style_table={
            'maxHeight':'450','overflowX': 'scroll',
        } ,
        editable=True,
        css=[{"selector": "table", "rule": "width: 100%;"}],
#         id=datatable_id
    )
    if df_in is None:
        df = pd.DataFrame({'no_data':['empty']})
    else:
        df = df_in.copy()
    dt.data=df.to_dict("rows")
    dt.columns=[{"name": i, "id": i,'editable': True if i in editable_columns else False} for i in df.columns.values]                    
    child_div = html.Div([html.H4(title,style={'height':'3px'}),dt])
    return child_div

def file_store_transformer(contents):
    if contents is None or len(contents)<=0 or contents[0] is None:
        d =  None
    else:
        d = dgrid.parse_contents(contents).to_dict('rows')
    return d


In [3]:
button_style={
    'line-height': '40px',
    'borderWidth': '1px',
    'borderStyle': 'dashed',
    'borderRadius': '1px',
    'textAlign': 'center',
    'background-color':'#fffff0',
    'vertical-align':'middle',
}
u1 = dcc.Upload(
            id='u1',
            children=html.Div(['enter a csv file']),
            # Allow multiple files to be uploaded
            multiple=False,
            style=button_style)

s1 = dcc.Store(id='s1')
s2 = dcc.Store(id='s2')

dt1 = html.Div([create_dt_div()],id='dt1')

app = dash.Dash()
app.layout = html.Div([u1,s1,s2,dt1])


In [4]:
# callback to move data from Upload to Store
@app.callback(
    [Output(s1.id,'data')], 
    [Input(u1.id,'contents')] 
)
def execute_callback1(*value):
    print(f'enter callback1 {value}')
    dict_df = None
    if value is not None and len(value)>0:
        dict_df = file_store_transformer(value[0])
    print(f'exit callback1 {dict_df}')
    return [dict_df]

# callback to move data from store to datatable
@app.callback(
#     [Output(dt1['dt_id'],'data'),Output(dt1['dt_id'],'columns')], 
    [Output(dt1.id,'children')], 
    [Input(s1.id,'data')]#  ,Input(s2.id,'data')] 
    )
def execute_callback2(*value):
    print(f'enter callback2 {value}')
    children = None
    if value is not None and len(value)>0:        
        df = pd.DataFrame(value[0])
        children = create_dt_div(df)
    print(f'exit callback2 {children}')
    return [children]



In [5]:
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_renderer/react-dom@16.8.6.min.js?v=1.0.0&m=1563025507 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_core_components/highlight.pack.js?v=1.0.0&m=1563025508 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET /_dash-component-suites/dash_table/bundle.js?v=4.0.1&m=1563025509 HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:07] "GET 

enter callback2 (None,)
exit callback2 Div([H4(children='Dash Table', style={'height': '3px'}), DataTable(columns=[], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[], editable=True, style_as_list_view=False, page_action='native', page_current=0, page_size=100, filter_action='none', sort_action='native', style_table={'maxHeight': '450', 'overflowX': 'scroll'}, style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': 'rgb(248, 248, 248)'}, {'if': {'column_id': 'symbol'}, 'textAlign': 'left'}, {'if': {'column_id': 'underlying'}, 'textAlign': 'left'}])])
enter callback1 (None,)
exit callback1 None
enter callback2 (None,)
exit callback2 Div([H4(children='Dash Table', style={'height': '3px'}), DataTable(columns=[], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[], editable=True, style_as_list_view=False, page_action='native', page_current=0, page_size=100, filter_action='none', sort_action='native', style_table={'maxHeight': '450', 'overflowX': 'scroll

127.0.0.1 - - [17/Jul/2019 15:23:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [17/Jul/2019 15:23:13] "POST /_dash-update-component HTTP/1.1" 200 -


enter callback1 ('data:text/csv;base64,c3ltYm9sLHBvc2l0aW9uDQpaU1kwMCwtNTANCl5FVVJVU0QsMjAwMDAwDQpYTFUsLTIwMA0KWExFLDIwMA0KVVNPLDINClNQWSwtMjAwDQpTT1lCLDM1MDANClpDWTAwLC01MA0KQ0xZMDAsNTAwDQpIT1kwMCwyMTAwMA0KSEdVMTksMjUwMDANCkNQRVIsLTI2MDANCkNUWjE5LDUwMA==',)
exit callback1 [{'symbol': 'ZSY00', 'position': -50}, {'symbol': '^EURUSD', 'position': 200000}, {'symbol': 'XLU', 'position': -200}, {'symbol': 'XLE', 'position': 200}, {'symbol': 'USO', 'position': 2}, {'symbol': 'SPY', 'position': -200}, {'symbol': 'SOYB', 'position': 3500}, {'symbol': 'ZCY00', 'position': -50}, {'symbol': 'CLY00', 'position': 500}, {'symbol': 'HOY00', 'position': 21000}, {'symbol': 'HGU19', 'position': 25000}, {'symbol': 'CPER', 'position': -2600}, {'symbol': 'CTZ19', 'position': 500}]
enter callback2 ([{'symbol': 'ZSY00', 'position': -50}, {'symbol': '^EURUSD', 'position': 200000}, {'symbol': 'XLU', 'position': -200}, {'symbol': 'XLE', 'position': 200}, {'symbol': 'USO', 'position': 2}, {'symbol': 'SPY', 'posi